In [1]:
import pandas as pd
df=pd.read_csv("homework.csv")
df

,X1,Y1,X2,Y2,Ca
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,1,1,0,0
4,1,1,1,1,1
...,...,...,...,...,...
34995,0,1,0,1,0
34996,1,1,1,1,1
34997,0,0,1,0,0
34998,0,0,0,1,0


# But : quel est l'impact de X2 sur Y1 ?

In [2]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

target="Y1"
evs={"X2":1}

In [3]:
learner=gum.BNLearner("homework.csv")
bn=learner.learnBN()
gnb.sideBySide(bn,gnb.getInference(bn),gnb.getInference(bn,evs={"X2":0}))

G <!-- X2 --> X2 X2 <!-- Y2 --> Y2 Y2 <!-- X2->Y2 --> X2->Y2 <!-- Y1 --> Y1 Y1 <!-- Y2->Y1 --> Y2->Y1 <!-- X1 --> X1 X1 <!-- X1->Y1 --> X1->Y1 <!-- Ca --> Ca Ca <!-- Ca->X2 --> Ca->X2 <!-- Ca->X1 --> Ca->X1


In [4]:
import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb

model=csl.CausalModel(bn)
cslnb.showCausalImpact(model,target, evs.keys())

In [6]:
learner=gum.BNLearner("homework.csv")
learner.useMIIC()
bn=learner.learnBN()
bn.reverseArc("X2","Ca") # Aie Aie Aie
gnb.sideBySide(bn,[(bn.variable(i).name(),bn.variable(j).name()) for (i,j) in learner.latentVariables()])

G <!-- X2 --> X2 X2 <!-- Y2 --> Y2 Y2 <!-- X2->Y2 --> X2->Y2 <!-- Y1 --> Y1 Y1 <!-- Y2->Y1 --> Y2->Y1 <!-- X1 --> X1 X1 <!-- X1->Y1 --> X1->Y1 <!-- Ca --> Ca Ca <!-- Ca->X2 --> Ca->X2 <!-- Ca->X1 --> Ca->X1,"[('Y2', 'Y1')]"


In [7]:
model=csl.CausalModel(bn,[("L1",("Y1","Y2"))])
cslnb.showCausalImpact(model,target, evs.keys())

# Model

In [8]:
bn=gum.fastBN("Ca->X1->Y1<-Pb")
bn.cpt("Ca").fillWith([0.8,0.2])
bn.cpt("Pb").fillWith([0.3,0.7])

bn.cpt("X1")[:]=[[0.9,0.1],[0.1,0.9]]

bn.cpt("Y1")[{"X1":0,"Pb":0}]=[0.8,0.2]
bn.cpt("Y1")[{"X1":1,"Pb":0}]=[0.2,0.8]
bn.cpt("Y1")[{"X1":0,"Pb":1}]=[0.6,0.4]
bn.cpt("Y1")[{"X1":1,"Pb":1}]=[0.4,0.6]

gnb.sideBySide(bn,bn.cpt("Ca"),bn.cpt("Pb"),bn.cpt("X1"),bn.cpt("Y1"))

In [9]:
bn.add("X2",2)
bn.add("Y2",2)
bn.addArc("X2","Y2")
bn.addArc("Ca","X2")
bn.addArc("Pb","Y2")

bn.cpt("X2").fillWith(bn.cpt("X1"),["X1","Ca"]) # copy cpt(X1) with the translation X2<-X1,Ca<-Ca
bn.cpt("Y2").fillWith(bn.cpt("Y1"),["Y1","X1","Pb"]) # copy cpt(Y1) with translation Y2<-Y1,X2<-X1,Pb<-Pb

gnb.sideBySide(bn,bn.cpt("X2"),bn.cpt("Y2"))

In [10]:
gnb.sideBySide(gnb.getInference(bn),gnb.getInference(bn,evs=evs))